# Time Series Analysis with Pandas

Reference: 
* http://pyvideo.org/scipy-2012/time-series-data-analysis-with-pandas.html (Wes McKinney)
* http://nbviewer.jupyter.org/github/pydata/pydata-book/blob/master/ch10.ipynb (Wes McKinney)

In this tutorial, we will delve into manipulating time series data in memory. We will describe common topics such as date arithmetic, alignment and join/merge methods, resampling and frequency conversion. 





### Date and Time Data Types and Tools

First, we need to describe how Python identifies TS datasets.

The datetime module supplies classes for manipulating dates and times. 

In [6]:
from datetime import datetime
now = datetime.now()
now

datetime.datetime(2017, 1, 29, 14, 40, 15, 602161)

In [7]:
now.year, now.month, now.day

(2017, 1, 29)

In [8]:
now?

In [9]:
delta = datetime(2016, 11, 7) - datetime(2016, 10, 7, 8, 15)
delta

datetime.timedelta(30, 56700)

In [10]:
print(delta.days, delta.seconds)

(30, 56700)


In [11]:
from datetime import timedelta

In [12]:
start = datetime(2011, 1, 7)
start + timedelta(12)

datetime.datetime(2011, 1, 19, 0, 0)

In [13]:
start - 2 * timedelta(12)

datetime.datetime(2010, 12, 14, 0, 0)

#### Converting between string and datetime

In [14]:
stamp = datetime(2011, 1, 3)

In [15]:
str(stamp)

'2011-01-03 00:00:00'

In [16]:
stamp.strftime('%Y-%m-%d')

'2011-01-03'

In [17]:
value = '2011-01-03'
datetime.strptime(value, '%Y-%m-%d')

datetime.datetime(2011, 1, 3, 0, 0)

In [18]:
datetime.strptime?

In [19]:
datestrs = ['7/6/2011', '8/6/2011']
[datetime.strptime(x, '%m/%d/%Y') for x in datestrs]

[datetime.datetime(2011, 7, 6, 0, 0), datetime.datetime(2011, 8, 6, 0, 0)]

In [20]:
from dateutil.parser import parse
parse('2011-01-03')

datetime.datetime(2011, 1, 3, 0, 0)

In [21]:
parse('Jan 31, 1997 10:45 PM')

datetime.datetime(1997, 1, 31, 22, 45)

In [22]:
parse('6/12/2011', dayfirst=True)

datetime.datetime(2011, 12, 6, 0, 0)

In [23]:
datestrs

['7/6/2011', '8/6/2011']

In [24]:
import pandas as pd

In [25]:
pd.to_datetime(datestrs)

DatetimeIndex(['2011-07-06', '2011-08-06'], dtype='datetime64[ns]', freq=None)

In [26]:
idx = pd.to_datetime(datestrs + [None])
idx 

DatetimeIndex(['2011-07-06', '2011-08-06', 'NaT'], dtype='datetime64[ns]', freq=None)

In [27]:
type(idx[2])

pandas.tslib.NaTType

In [28]:
pd.isnull(idx)

array([False, False,  True], dtype=bool)

# Time Series Basics

In [29]:
import numpy as np

In [30]:
from datetime import datetime

dates = [datetime(2011, 1, 2), datetime(2011, 1, 5), datetime(2011, 1, 7),
        datetime(2011, 1, 8), datetime(2011, 1, 10), datetime(2011, 1, 12)]
ts = pd.Series(np.random.randn(6), index=dates)
ts

2011-01-02    0.528335
2011-01-05    0.423362
2011-01-07    0.576798
2011-01-08   -0.745457
2011-01-10   -1.488438
2011-01-12   -0.084843
dtype: float64

In [31]:
type(ts)


pandas.core.series.Series

In [32]:
ts.index

DatetimeIndex(['2011-01-02', '2011-01-05', '2011-01-07', '2011-01-08',
               '2011-01-10', '2011-01-12'],
              dtype='datetime64[ns]', freq=None)

In [33]:
stamp = ts.index[0]
stamp
# note: output changed from <Timestamp: 2011-01-02 00:00:00> to Timestamp('2011-01-02 00:00:00')


Timestamp('2011-01-02 00:00:00')

# Indexing, Selection, Subsetting

In [34]:
stamp = ts.index[2]
ts[stamp]

0.57679767028502082

In [35]:
ts['1/10/2011']

-1.4884376061359976

In [36]:
ts['20110110']

-1.4884376061359976

In [37]:
longer_ts = pd.Series(np.random.randn(1000),
                   index=pd.date_range('1/1/2000', periods=1000))
longer_ts

2000-01-01    0.025930
2000-01-02    0.320209
2000-01-03    0.227521
2000-01-04   -0.510986
2000-01-05   -0.104024
2000-01-06   -0.948414
2000-01-07    0.094924
2000-01-08    0.538624
2000-01-09   -0.361278
2000-01-10    1.111608
2000-01-11    1.344681
2000-01-12   -0.698914
2000-01-13   -0.698168
2000-01-14   -0.856511
2000-01-15   -0.643365
2000-01-16    0.065090
2000-01-17    0.624272
2000-01-18    0.380858
2000-01-19    0.267747
2000-01-20   -0.830849
2000-01-21    0.924632
2000-01-22    1.102501
2000-01-23    1.200293
2000-01-24    1.300202
2000-01-25   -0.177385
2000-01-26   -0.916747
2000-01-27    0.583171
2000-01-28   -0.885461
2000-01-29    0.751219
2000-01-30    0.015034
                ...   
2002-08-28    0.031103
2002-08-29   -1.187072
2002-08-30   -0.167556
2002-08-31   -0.291839
2002-09-01   -0.537447
2002-09-02    1.171958
2002-09-03   -1.160130
2002-09-04   -0.569229
2002-09-05   -1.051379
2002-09-06   -0.021112
2002-09-07    0.422147
2002-09-08    0.408175
2002-09-09 

In [38]:
longer_ts['2001']

2001-01-01    1.035696
2001-01-02    1.055234
2001-01-03   -0.285427
2001-01-04    1.238867
2001-01-05   -0.279783
2001-01-06   -0.598748
2001-01-07   -0.490526
2001-01-08    0.571936
2001-01-09   -0.099511
2001-01-10   -1.594836
2001-01-11    1.554262
2001-01-12    0.923525
2001-01-13   -0.411915
2001-01-14   -1.093963
2001-01-15    1.494383
2001-01-16   -1.971920
2001-01-17    1.778201
2001-01-18    0.124129
2001-01-19   -2.455421
2001-01-20   -0.769413
2001-01-21    0.086626
2001-01-22   -0.022375
2001-01-23   -0.967111
2001-01-24   -0.631183
2001-01-25   -1.595324
2001-01-26    0.980611
2001-01-27    2.095155
2001-01-28    0.306110
2001-01-29    0.190122
2001-01-30   -0.744256
                ...   
2001-12-02    0.693386
2001-12-03    0.209543
2001-12-04    0.111973
2001-12-05   -0.839984
2001-12-06    0.904468
2001-12-07    0.386202
2001-12-08    0.584379
2001-12-09    0.848045
2001-12-10    3.081312
2001-12-11    3.532269
2001-12-12   -0.447755
2001-12-13    0.643428
2001-12-14 

In [39]:
longer_ts['2001-05']

2001-05-01   -1.677450
2001-05-02   -1.945501
2001-05-03   -1.055870
2001-05-04    0.862207
2001-05-05   -0.113309
2001-05-06   -0.275244
2001-05-07   -0.073101
2001-05-08   -1.036216
2001-05-09   -0.997204
2001-05-10   -0.596772
2001-05-11   -1.026563
2001-05-12   -0.115698
2001-05-13   -1.255741
2001-05-14   -0.608076
2001-05-15   -0.798250
2001-05-16   -1.060590
2001-05-17    0.048513
2001-05-18    0.999374
2001-05-19   -0.720583
2001-05-20    1.197031
2001-05-21   -0.370651
2001-05-22    0.888532
2001-05-23    0.791822
2001-05-24    0.309837
2001-05-25   -0.918928
2001-05-26   -0.911078
2001-05-27    0.574433
2001-05-28   -0.870334
2001-05-29   -0.630038
2001-05-30   -0.097926
2001-05-31    0.184903
Freq: D, dtype: float64

In [40]:
ts[datetime(2011, 1, 7):]

2011-01-07    0.576798
2011-01-08   -0.745457
2011-01-10   -1.488438
2011-01-12   -0.084843
dtype: float64

In [41]:
ts['1/6/2011':'1/11/2011']



2011-01-07    0.576798
2011-01-08   -0.745457
2011-01-10   -1.488438
dtype: float64

In [42]:
ts

2011-01-02    0.528335
2011-01-05    0.423362
2011-01-07    0.576798
2011-01-08   -0.745457
2011-01-10   -1.488438
2011-01-12   -0.084843
dtype: float64

In [43]:
ts.truncate(after='1/9/2011')

2011-01-02    0.528335
2011-01-05    0.423362
2011-01-07    0.576798
2011-01-08   -0.745457
dtype: float64

In [44]:
dates = pd.date_range('1/1/2000', periods=100, freq='W-WED')
long_df = pd.DataFrame(np.random.randn(100, 4),
                    index=dates,
                    columns=['Colorado', 'Texas', 'New York', 'Ohio'])
long_df.ix['5-2001']

,Colorado,Texas,New York,Ohio
2001-05-02,-0.519105,-0.840564,-0.453521,-0.302537
2001-05-09,-2.460674,0.726575,-0.390843,1.821822
2001-05-16,0.318574,-1.166980,-1.209197,0.062502
2001-05-23,1.914947,-0.433130,0.566289,1.095055
2001-05-30,-0.329925,-0.323777,1.151928,-0.300726


# Time Series with Duplicate Indices

In [45]:
dates = pd.DatetimeIndex(['1/1/2000', '1/2/2000', '1/2/2000', '1/2/2000', '1/3/2000'])
dups_ts = pd.Series(np.arange(5), index=dates)
dups_ts

2000-01-01    0
2000-01-02    1
2000-01-02    2
2000-01-02    3
2000-01-03    4
dtype: int64

In [46]:
dups_ts.index.is_unique

False

In [47]:
dups_ts['1/3/2000']

4

In [48]:
dups_ts['1/2/2000']

2000-01-02    1
2000-01-02    2
2000-01-02    3
dtype: int64

In [49]:
grouped = dups_ts.groupby(level=0)
grouped.mean()

2000-01-01    0
2000-01-02    2
2000-01-03    4
dtype: int64

In [50]:
grouped.count()


2000-01-01    1
2000-01-02    3
2000-01-03    1
dtype: int64

#### Date Ranges, Frequencies, and Shifting

In [51]:
ts

2011-01-02    0.528335
2011-01-05    0.423362
2011-01-07    0.576798
2011-01-08   -0.745457
2011-01-10   -1.488438
2011-01-12   -0.084843
dtype: float64

In [53]:
ts.resample('D').sum()

2011-01-02    0.528335
2011-01-03         NaN
2011-01-04         NaN
2011-01-05    0.423362
2011-01-06         NaN
2011-01-07    0.576798
2011-01-08   -0.745457
2011-01-09         NaN
2011-01-10   -1.488438
2011-01-11         NaN
2011-01-12   -0.084843
Freq: D, dtype: float64

#### Resampling and Frequency Conversion

In [55]:
import numpy as np

In [61]:
rng = pd.date_range('1/1/2000', periods=100, freq='D')
ts = pd.Series(np.random.randn(len(rng)), index = rng)
ts.resample('M', kind='period').mean()

2000-01    0.173096
2000-02   -0.262413
2000-03   -0.506827
2000-04    0.689774
Freq: M, dtype: float64

##### Downsampling

In [64]:
rng = pd.date_range('1/1/2000', periods=12, freq='T')
ts = pd.Series(np.arange(12), index=rng)
ts

2000-01-01 00:00:00     0
2000-01-01 00:01:00     1
2000-01-01 00:02:00     2
2000-01-01 00:03:00     3
2000-01-01 00:04:00     4
2000-01-01 00:05:00     5
2000-01-01 00:06:00     6
2000-01-01 00:07:00     7
2000-01-01 00:08:00     8
2000-01-01 00:09:00     9
2000-01-01 00:10:00    10
2000-01-01 00:11:00    11
Freq: T, dtype: int64

In [66]:
ts.resample('5min').sum()

2000-01-01 00:00:00    10
2000-01-01 00:05:00    35
2000-01-01 00:10:00    21
Freq: 5T, dtype: int64

In [67]:
ts.resample('5min', closed='left').sum()

2000-01-01 00:00:00    10
2000-01-01 00:05:00    35
2000-01-01 00:10:00    21
Freq: 5T, dtype: int64

In [68]:
ts.resample('5min', closed='left', label='left').sum()

2000-01-01 00:00:00    10
2000-01-01 00:05:00    35
2000-01-01 00:10:00    21
Freq: 5T, dtype: int64

In [70]:
ts.resample('5min', loffset='-1s').sum()

1999-12-31 23:59:59    10
2000-01-01 00:04:59    35
2000-01-01 00:09:59    21
Freq: 5T, dtype: int64

##### Open-High-Low-Close (OHLC) resampling

###### Question: What is an application of this function 

In [73]:
ts.resample('5min').ohlc()

,open,high,low,close
2000-01-01 00:00:00,0,4,0,4
2000-01-01 00:05:00,5,9,5,9
2000-01-01 00:10:00,10,11,10,11
